# Data exploration and preprocessing

- In this notebook, we load in the MRI scans and their segmentations, build a Dataset object for the train and test set.
- Then we check some basic stats of the datasets and visualise a few scans.
- Finally, we carry out our preprocessing steps and save the train and test datasets.

In [22]:
%load_ext autoreload
%autoreload 2

import os
import copy
import logging
from glob import glob
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append('../src/')

from data_utils import Dataset


# get TF logger - set it to info for more tracking process
log = logging.getLogger('pytorch')
log.setLevel(logging.WARNING)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Create train and test datasets

Find all the raw data files and build train and test Dataset objects from the patients' scans. 

In [ ]:
# get file paths to DICOM MRI scans and segmentation images
# note that leaderboard samples can be used for training
train_scan_files = glob('../data/raw/train/**/*.dcm', recursive=True)
train_scan_files += glob('../data/raw/leaderboard/**/*.dcm', recursive=True)
test_scan_files = glob('../data/raw/test/**/*.dcm', recursive=True)

# ProstateDx-01-0006_corrected_label.nrrd was renamed to ProstateDx-01-0006.nrrd
# In the leaderboard and test folders the _truth postfix have been removed from all nrrd files
train_seg_files = glob('../data/raw/train/**/*.nrrd', recursive=True)
train_seg_files += glob('../data/raw/leaderboard/**/*.nrrd', recursive=True)
test_seg_files = glob('../data/raw/test/**/*.nrrd', recursive=True)

# build datasets from file paths
train_dataset = Dataset(scan_files=train_scan_files, seg_files=train_seg_files)
test_dataset = Dataset(scan_files=test_scan_files, seg_files=test_seg_files)

## Check basic stats
Check number of patients in train and test datasets.

In [ ]:
train_n = len(train_dataset.patient_ids)
test_n = len(test_dataset.patient_ids)
train_scan_nums = [p.scans.shape[0] for p in train_dataset.patients.values()]
test_scan_nums = [p.scans.shape[0] for p in test_dataset.patients.values()]

print('Number of patients in train dataset: %d' % train_n)
print('Number of patients in test dataset: %d' % test_n)
print('Number of scans in train dataset: %d' % sum(train_scan_nums))
print('Number of scans in test dataset: %d' % sum(test_scan_nums))

Check distribution of number of scans in train and test datasets.
- They both seem bi-modal with roughly the same peaks. 

In [ ]:
fig, ax = plt.subplots(1, 2)
ax[0].set_title('# scans in train dataset')
ax[0].hist(train_scan_nums, bins=10)
ax[1].set_title('# scans in test dataset')
ax[1].hist(test_scan_nums, bins=10)

Make sure that none of the patients have scans from mixed manufacturers and with mixed slice thickness.

In [ ]:
# extract manufacturer and thickness sets from each patient
train_manufacturers = [p.manufacturers for p in train_dataset.patients.values()]
train_thicknesses = [p.thicknesses for p in train_dataset.patients.values()]
test_manufacturers = [p.manufacturers for p in test_dataset.patients.values()]
test_thicknesses = [p.thicknesses for p in test_dataset.patients.values()]

# check if any patient has slices from two different manufacturers or thicknesses - NO
for m in train_manufacturers + test_manufacturers:
    assert len(m) == 1

for t in train_thicknesses + test_thicknesses:
    assert len(t) == 1

### Create summary table 

Collate all information into a pandas DataFrame from the datasets so we can analyse it easily later.

In [ ]:
# collapse all list of sets to simple list
train_manufacturers = [list(i)[0] for i in train_manufacturers]
train_thicknesses = [list(i)[0] for i in train_thicknesses]
test_manufacturers = [list(i)[0] for i in test_manufacturers]
test_thicknesses = [list(i)[0] for i in test_thicknesses]

# extract scan width, height and max value
train_widths = [p.scans.shape[1] for p in train_dataset.patients.values()]
train_heights = [p.scans.shape[2] for p in train_dataset.patients.values()]
train_max = [p.scans.max() for p in train_dataset.patients.values()]
test_widths = [p.scans.shape[1] for p in test_dataset.patients.values()]
test_heights = [p.scans.shape[2] for p in test_dataset.patients.values()]
test_max = [p.scans.max() for p in test_dataset.patients.values()]

# calculate contingency table from them
df_summary = pd.DataFrame(
    list(
        zip(
            train_dataset.patient_ids + test_dataset.patient_ids,
            train_manufacturers + test_manufacturers,
            train_thicknesses + test_thicknesses,
            train_widths + test_widths,
            train_heights + test_heights,
            train_max + test_max,
            train_scan_nums + test_scan_nums,
            ['train'] * train_n + ['test'] * test_n
        )
    ), 
    columns = ['patient_id', 'manufacturer', 'thickness', 
               'width', 'heigth', 'max_val', 'scan_num', 'dataset']
)

- Looks like the test and train datasets have been properly stratified with respect to manufacturer, i.e. half of the sample are from Siemens and half of them are from Philips.
- However, the test dataset doesn't have slices of 4mm thickness.

In [ ]:
df_summary.drop(
    ['width', 'heigth', 'scan_num', 'max_val'], axis=1
).groupby(
    ['dataset', 'manufacturer', 'thickness']
).count()

- Philips is the higher resolution machine, all scans are rectangular.

In [ ]:
df_summary.drop(
    ['thickness', 'max_val', 'scan_num'], axis=1
).groupby(
    ['dataset', 'manufacturer', 'width', 'heigth']
).count()

- There's a large variation in the scans' maximum values and there are some clear outliers too (Siemens scan with max=65283, coming from the 18th scan of patient Prostate3T-01-0018)

In [ ]:
df_summary.drop(
    ['thickness', 'patient_id', 'scan_num'], axis=1
).groupby(
    ['dataset', 'manufacturer', 'width', 'heigth']
).agg(['min', 'max', 'mean'])

- Here's the reason for those bi-modal histograms above, Philips mave higher number of scans on average.

In [ ]:
df_summary.drop(
    ['thickness', 'max_val', 'width', 'heigth', 'patient_id'], axis=1
).groupby(
    ['dataset', 'manufacturer']
).agg(['min', 'max', 'median'])

## Visualise scans

Each patient's scans can be viewed as an animation or as a tiled figure. Let's have a look at some of these.

**Note** you'll need to re-execute the cell to watch the animation.

In [ ]:
# animation
patient_id = train_dataset.patient_ids[23]
train_dataset.patients[patient_id].patient_anim_scans()

In [ ]:
# tiled figure
patient_id = train_dataset.patient_ids[2]
train_dataset.patients[patient_id].patient_tile_scans()

## Calculate class frequency

The 3 classes are imbalanced, calculate their frequency in the data. This will inform our weighting scheme that we use with the loss function at training time.

In [ ]:
class_freq = np.zeros(3)
for i in range(len(train_dataset.patients.keys())):
    patient_id = train_dataset.patient_ids[i]
    seg = train_dataset.patients[patient_id].seg
    class0 = np.count_nonzero(seg == 0)
    class1 = np.count_nonzero(seg == 1)
    class2 = np.count_nonzero(seg == 2)
    class_freq += np.array([class0, class1, class2])
class_freq = class_freq / class_freq.sum()
inv_class_freq = 1/class_freq
norm_inv_class_freq = inv_class_freq / inv_class_freq.sum()
norm_inv_class_freq


## Preprocess data

As we've seen from the summary stats above the scans are non-normalised and span a wide range of maximal values, resolution and number of scans. 

The `preprocess_dataset` method:
  - normalises the scans to be between zero and one
  - resizes (and downsample) each scan and target segmentation image to the same width and height (128, 128)
    - since 3D U-Net is fully convolutional, this isn't needed necessarily but it reduces the required memory at training time
  - cap depth of scans, i.e. the number of scans across the patients: <32
    - this ensures that with a 4 layer deep 3D U-Net we'll get matching dimensions when we concatenate the shurtcuts in the network 
    - we discard extra scans (i.e. more 32) and patients with less will be padded with zeros by TensorFlow

### First, save preprocessed but non-resized test dataset

The performance evaluation has to be done ono original (i.e. non rescaled images). 

In [ ]:
test_dataset_non_resized = copy.deepcopy(test_dataset)
test_dataset_non_resized.preprocess_dataset(resize=False, width=_, height=_, max_scans=32)
test_dataset_non_resized.save_dataset('../data/processed/test_dataset.pckl')

Now let's resize and preprocess both train and test.

In [ ]:
train_dataset.preprocess_dataset(resize= True,width=128, height=128, max_scans=32)
test_dataset.preprocess_dataset(resize= True,width=128, height=128, max_scans=32)

Let's check if the scans and targets still look reasonable on the previous tiled example.

In [ ]:
# note the target is now a one-hot tensor, so we only show the 2nd class
patient_id = train_dataset.patient_ids[2]
train_dataset.patients[patient_id].patient_tile_scans()

Finally, let's check that the preprocessing worked for all images. Specifically we should find that:
- the number of scans are 32 or less for all patients
- the resolution is 128 by 128
- the maximum value of the scans is less than or equal to 1
- the corresponding target tensor has an extra dimension, corresponding to the one hot encoding of the 3 classes

In [ ]:
datasets = [train_dataset, test_dataset]
for dataset in datasets:
    for i in range(len(dataset.patients.keys())):
        patient_id = dataset.patient_ids[i]
        scans = dataset.patients[patient_id].scans 
        seg = dataset.patients[patient_id].seg

        assert(scans.shape[1:] == (128, 128))
        assert(scans.shape[0] <= 32)
        assert(scans.max() <= 1)

        assert(seg.shape[1:3] == (128, 128))
        assert(seg.shape[0] <= 32)
        assert(seg.shape[3] == 3)

## Save resized datasets

We save them as pickled objects so we can use them later for training and model evaluation.

In [ ]:
train_dataset.save_dataset('../data/processed/train_dataset_resized.pckl')
test_dataset.save_dataset('../data/processed/test_dataset_resized.pckl')